In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import OceanBase
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import CharacterTextSplitter

dashscope_api = '<YOUR-API-KEY>'
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=dashscope_api
)
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# print(query_result)

In [2]:
loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [3]:
DEMO_TABLE_NAME = "demo_ann"
ob = OceanBase.from_documents(
    documents=docs,
    embedding=embeddings,
    table_name=DEMO_TABLE_NAME,
    connection_args={
        "host": "<YOUR-HOST>",
        "port": "<YOUR-PORT>",
        "user": "<YOUR-USER>",
        "password": "<YOUR-PASSWORD>",
        "db_name": "<YOUR-DBNAME>",
    },
    drop_old=True,
)

/Users/dimdew/Documents/code/pyobvector/pyobvector/client/ob_vec_client.py:245: SAWarning: Unknown schema content: '  VECTOR KEY `vidx` (`embedding`) WITH (DISTANCE=L2,M=16,EF_CONSTRUCTION=256,LIB=VSAG,TYPE=HNSW, EF_SEARCH=64) BLOCK_SIZE 16384'
  table = Table(table_name, self.metadata_obj, autoload_with=self.engine)


In [4]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = ob.similarity_search_with_score(query, k=3)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  96.55317499046893
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
------------------------